In [1]:
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import epiweeks
import geopandas as gpd
import altair as alt

dfs = pd.read_csv('../macro_saude.csv')


In [2]:
code_to_state = {33: 'RJ', 32: 'ES', 41: 'PR', 23: 'CE', 21: 'MA',
 31: 'MG', 42: 'SC', 26: 'PE', 25: 'PB', 24: 'RN', 22: 'PI', 27: 'AL',
 28: 'SE', 35: 'SP', 43: 'RS', 15: 'PA', 16: 'AP', 14: 'RR',  11: 'RO',
 13: 'AM', 12: 'AC', 51: 'MT', 50: 'MS', 52: 'GO', 17: 'TO', 53: 'DF',
 29: 'BA'}



In [3]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
df_states = gpd.read_file('../states.gpkg')

df_states.head()

/opt/homebrew/Caskroom/miniforge/base/envs/alerta_dengue/lib/python3.9/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


,CD_UF,NM_UF,SIGLA,NM_REGIAO,geometry
0,11,Rondônia,RO,Norte,"POLYGON ((-62.86662 -7.97587, -62.86017 -7.982..."
1,12,Acre,AC,Norte,"POLYGON ((-73.16550 -7.34166, -73.05480 -7.381..."
2,13,Amazonas,AM,Norte,"POLYGON ((-67.32609 2.02971, -67.31682 2.00125..."
3,14,Roraima,RR,Norte,"POLYGON ((-60.20051 5.26434, -60.19828 5.26045..."
4,15,Pará,PA,Norte,"MULTIPOLYGON (((-46.43676 -1.01905, -46.43723 ..."


In [7]:
for i, m in enumerate(glob.glob('./forecast_tables/forecast_*.csv.gz')):
    if i == 0:
        df = pd.read_csv(m)
        dates = df.date.unique()
    else:
        df = pd.concat([df, pd.read_csv(m)])

df.prob_low = -df.prob_low
df['prob_color'] = df.apply(lambda x: x.prob_low if abs(x.prob_low) > abs(x.prob_high) else x.prob_high, axis=1)
df['prob_color'] = df.prob_color.apply(lambda x: 0 if abs(x) < 50 else x)

df_macro = gpd.read_file('./shapefile_macro.gpkg')

 
df_macro = df_macro.merge(df, left_on='code_macro', right_on='macroregion', how='left')

df_macro.head()

,code_macro,name_code_macro,name_macro,geometry,Unnamed: 0,date,lower_2_5,lower_25,forecast,upper_75,upper_97_5,macroregion,prob_high,prob_low,HT,LT,HTinc,LTinc,prob_color
0,1101.0,Cacoal - 1101,Cacoal,"POLYGON ((-62.11971 -13.15353, -62.12429 -13.1...",0,2024-01-28,6.206557,35.228528,44.985627,61.932060,99.825539,1101,0.0,-73.0,203.599542,60.929516,39.169125,11.721813,-73.0
1,1101.0,Cacoal - 1101,Cacoal,"POLYGON ((-62.11971 -13.15353, -62.12429 -13.1...",1,2024-02-04,10.068197,37.552025,50.060446,71.803508,99.497638,1101,0.0,-60.0,229.359452,57.074040,44.124897,10.980084,-60.0
2,1101.0,Cacoal - 1101,Cacoal,"POLYGON ((-62.11971 -13.15353, -62.12429 -13.1...",2,2024-02-11,19.669612,45.282510,57.103748,80.527496,111.759103,1101,0.0,-59.0,250.017985,63.877479,48.099252,12.288952,-59.0
3,1101.0,Cacoal - 1101,Cacoal,"POLYGON ((-62.11971 -13.15353, -62.12429 -13.1...",3,2024-02-18,22.782712,46.631853,59.633913,81.625677,121.894206,1101,0.0,-55.0,225.307843,63.426117,43.345436,12.202117,-55.0
4,1101.0,Cacoal - 1101,Cacoal,"POLYGON ((-62.11971 -13.15353, -62.12429 -13.1...",4,2024-02-25,21.500165,47.199570,60.925656,83.851711,115.257969,1101,0.0,-45.0,224.817237,56.981611,43.251052,10.962303,0.0


In [8]:
df_macro.shape

(11170, 19)

In [9]:
df_macro['desc_prob'] = np.nan

df_macro.loc[df_macro.prob_color > 0, 'desc_prob'] =   'Probabilidade de a incidência nesta data estar acima do limiar superior histórico'

df_macro.loc[df_macro.prob_color < 0, 'desc_prob'] =   'Probabilidade de a incidência nesta data estar abaixo do limiar inferior histórico'



In [11]:
select_date = alt.selection_point(
    name="date",
    fields=["date"],
    bind = alt.binding_select(options=df_macro.date.unique(), name='date'),
    value='2024-01-28',
)

map_dist = alt.Chart(df_macro, title='').mark_geoshape().encode(
    color= alt.Color('HTinc:Q',
              scale=alt.Scale(scheme='Viridis'),
             legend=alt.Legend(direction='vertical', orient='right', legendY=30, 
        title = 'Incidência /100.000 hab.', titleOrient = 'left')), 
    
    tooltip = [alt.Tooltip('name_code_macro:N', title='Macrorregião:'),
               alt.Tooltip('HTinc:Q', title='Incidência:')]).add_params(select_date).transform_filter(select_date)


text_dist = alt.Chart(df_macro).mark_text(dy=-170, dx = 20, size=14).encode(
    text='date:N'
).transform_filter(
    select_date
).transform_calculate(date = '"Limiar superior de Incidência na semana de " + datum.date')

#states_line = alt.Chart(df_states).mark_geoshape(
 #   filled=False,
  #  strokeWidth=1.5,
   # color = 'black'
#)

map_prob = alt.Chart(df_macro, title='').mark_geoshape().encode(
    color= alt.Color('prob_color:Q',
              scale=alt.Scale(scheme='redblue',reverse=True),
                    legend=alt.Legend(direction='vertical', orient='right', legendY=30, 
        title = 'Probabilidade (%)', titleOrient = 'left')),
    tooltip = [alt.Tooltip('name_code_macro:N', title='Macrorregião:'),
               alt.Tooltip('prob_color:Q', title='Probabilidade (%):'),
              alt.Tooltip('desc_prob:N', title = 'Info:')]
).add_params(select_date).transform_filter(select_date)


text_prob = alt.Chart(df_macro).mark_text(dy=-170, dx = 20, size=14).encode(
    text='date:N'
).transform_filter(
    select_date
).transform_calculate(date = '"Previsão probabilística na semana de " + datum.date')



final_maps = alt.hconcat(alt.layer(map_dist, text_dist), alt.layer(map_prob,text_prob)).resolve_scale(color='independent')

'''
# Base chart for data tables
ranked_table_prob = alt.Chart(df_muni[['date', 'name_code_macro', 'prob_color', 'HTinc']]).mark_text(align='right').encode(
    y=alt.Y('row_number:O').axis(None)
).add_params(select_date).transform_filter(
    select_date
).transform_window(
    row_number='row_number()',
    rank='rank(prob_color)',
    sort=[alt.SortField('prob_color', order='descending')]
    
).transform_filter(
    alt.datum.rank < 15
)


# Data Tables
d = ranked_table_prob.encode(text='date:N').properties(
    title=alt.Title(text='date', align='right')
)
name = ranked_table_prob.encode(text='name_code_macro:N').properties(
    title=alt.Title(text='Macrorregião', align='right')
)
prob = ranked_table_prob.encode(text='prob_color:N').properties(
    title=alt.Title(text='Probabilidade (%)', align='right')
)

inc =  ranked_table_prob.encode(text='HTinc:Q').properties(
    title=alt.Title(text='Limiar superior de Incidência (100k)', align='right')
)

table_prob = alt.hconcat(d, name, inc, prob) # Combine data tables

#table_prob = table_prob.configure_view(
 #   stroke=None
#)


#final_plot = alt.vconcat(final_maps, table_prob).configure_view(
 #  stroke=None)
'''

"\n# Base chart for data tables\nranked_table_prob = alt.Chart(df_muni[['date', 'name_code_macro', 'prob_color', 'HTinc']]).mark_text(align='right').encode(\n    y=alt.Y('row_number:O').axis(None)\n).add_params(select_date).transform_filter(\n    select_date\n).transform_window(\n    row_number='row_number()',\n    rank='rank(prob_color)',\n    sort=[alt.SortField('prob_color', order='descending')]\n    \n).transform_filter(\n    alt.datum.rank < 15\n)\n\n\n# Data Tables\nd = ranked_table_prob.encode(text='date:N').properties(\n    title=alt.Title(text='date', align='right')\n)\nname = ranked_table_prob.encode(text='name_code_macro:N').properties(\n    title=alt.Title(text='Macrorregião', align='right')\n)\nprob = ranked_table_prob.encode(text='prob_color:N').properties(\n    title=alt.Title(text='Probabilidade (%)', align='right')\n)\n\ninc =  ranked_table_prob.encode(text='HTinc:Q').properties(\n    title=alt.Title(text='Limiar superior de Incidência (100k)', align='right')\n)\n\ntab

display(final_maps)

In [ ]:
#final_maps.save('map_macro.html',  embed_options={'renderer':'svg'})

display(final_plot)